# Sprint 深層学習スクラッチ リカレントニューラルネットワーク

## 1.このSprintについて

### Sprintの目的
- スクラッチを通してリカレントニューラルネットワークの基礎を理解する

### どのように学ぶか
- スクラッチでリカレントニューラルネットワークの実装を行います。

## 2.リカレントニューラルネットワークスクラッチ

リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。


クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

## 【まとめ】最終コード

In [ ]:
import numpy as np

### ■ ScratchSimpleRNNClassifier:

In [ ]:
class ScratchSimpleRNNClassifier:
    def __init__(self):
        pass


    def fit(self, X, sigma=0.01, lr=0.01, epochs=1):
        """
        X : (batch_size, n_sequences, n_features)
        out : (batch_size, n_sequences, n_nodes)
        """
        self.sigma = sigma
        self.lr = lr
        self.epochs = epochs
        
        self.n_samples = X.shape[0]
        self.n_sequences = X.shape[1]
        self.n_features = X.shape[2]
        self.n_nodes = 4   # 任意値

        self.activation = Tanh()
        self.rnn = MyRNN(input_seq=self.n_sequences,
                         input_features=self.n_features,
                         initializer=HeInitializer(self.sigma, self.n_features, self.n_nodes),
                         optimizer=SGD(lr=self.lr),
                         activation=self.activation)

        # 本来はミニバッチ毎に実行
        out = self.forward(X)
        return out


    def forward(self, inputs):
        x = self.rnn.forward(inputs)
        return x


    def backward(self, dA):
        dZ = self.rnn.backward(dA)
        return dZ


    def predict(self):
        pass


### ■ MyRNN

In [26]:
class MyRNN():
    def __init__(self, input_seq, input_features, initializer, optimizer, activation):
        self.optimizer = optimizer
        self.activation = activation
        # w,bの初期化
        self.Wx = initializer.W()
        self.B = initializer.B()

        # 各サイズ
        self.batch_size = None
        self.n_sequences = input_seq
        self.n_features = input_features
        self.n_nodes = self.Wx.shape[-1]

        # 隠れ状態ベクトルの重み
        self.Wh = initializer.W(self.n_nodes, self.n_nodes)

        # AdaGrad用
        self.H_W = np.zeros([self.Wx.shape[0], self.Wx.shape[1]])
        self.H_B = np.zeros(self.B.shape[1])

        # ※問題2に合わせてパラメータを上書き！！
        self.Wx = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
        self.Wh = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
        self.n_nodes = self.Wx.shape[1] # 4
        # h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
        self.B = np.array([[1, 1, 1, 1]]).astype('float') # (n_nodes,)


    def forward(self, X):
        self.X = X
        self.batch_size = X.shape[0]
        self.S = np.zeros((self.batch_size, self.n_sequences, self.n_nodes))

        for t in range(self.n_sequences):
            self.S[:, t, :] = self.activation(X[:, t, :]@self.Wx + self.S[:, t-1, :]@self.Wh + self.B)
        return self.S


    def backward(self, dA):
        dHt_1 = np.zeros((self.batch_size, 1, self.n_nodes))
        self.dWx = np.zeros((1, self.n_features, self.n_nodes))
        self.dWh = np.zeros((1, self.n_nodes, self.n_nodes))
        self.dB = np.zeros((1, 1, self.n_nodes))
        dXt = np.zeros((self.batch_size, self.n_sequences, self.n_features))

        for t in range(self.n_sequences-1, -1, -1):
            dHt = dA[:, t, :] + dHt_1    # 前の時刻からの状態の誤差と出力の誤差の合計
            print('■ dHt_1 ----------------------------------\n', dHt_1)
            print('■ dHt ----------------------------------\n', dHt)
            print('■ dA ----------------------------------\n', dA)

            dAt = dHt * (1 - np.tanh(dA[:, t, :])**2)
            print('■ dAt ----------------------------------\n', dAt)
            
            self.dWx += self.X[:, t, :].T @ dAt
            print('■ dWx ----------------------------------\n', self.dWx)
            self.dB += dAt

            self.dWh += self.S[:, t, :].T @ dAt
            print('■ self.dWh ----------------------------------\n', self.dWh)

            dHt_1 = dAt @ self.Wh.T
            dXt[:, t:t+1, :] += dAt @ self.Wx.T
            print('■ dHt_1 ----------------------------------\n', dHt_1)
            print('■ dXt ----------------------------------\n', dXt)
        
        # 蓄積した各パラメータの形状を整える
        self.dWx = self.dWx.squeeze()
        self.dWh = self.dWh.squeeze()
        self.dB = self.dB.squeeze()

        # 更新
        self.optimizer.rnn_update(self)
        return dXt


### ■ SimpleInitializer

In [ ]:
class SimpleInitializer:
    def __init__(self, sigma, n_nodes1, n_nodes2):
        self.sigma = sigma
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.mean = 0
        self.s = 1

    def W(self, n_nodes1=None, n_nodes2=None):
        if np.any(n_nodes1 != None):
            self.n_nodes1 = n_nodes1
        if np.any(n_nodes2 != None):
            self.n_nodes2 = n_nodes2

        W = self.sigma * np.random.normal(loc=self.mean, scale=self.s, size=(self.n_nodes1, self.n_nodes2))
        return W


    def B(self, num=None):
        if num == None:
            # B = self.sigma * np.random.normal(loc=self.mean, scale=self.s, size=(1, self.n_nodes2))
            B = np.zeros([1, self.n_nodes2])
        else:
            B = np.zeros([1, num])
        return B

class XavierInitializer(SimpleInitializer):
    def __init__(self, sigma, n_nodes1, n_nodes2):
        super().__init__(sigma, n_nodes1, n_nodes2)
        self.sigma = 1
        self.s = 1 / np.sqrt(n_nodes1)

class HeInitializer(SimpleInitializer):
    def __init__(self, sigma, n_nodes1, n_nodes2):
        super().__init__(sigma, n_nodes1, n_nodes2)
        self.sigma = 1
        self.s = np.sqrt(2 / self.n_nodes1)


### ■ Optimizer

In [25]:
class SGD:
    def __init__(self, lr):
        self.lr = lr

    def rnn_update(self, layer):
        layer.Wx -= self.lr * layer.dWx / layer.batch_size
        layer.Wh -= self.lr * layer.dWh / layer.batch_size
        layer.B -= self.lr * layer.dB / layer.batch_size
        return layer.Wx, layer.Wh

### ■ Activation

In [ ]:
class Softmax:
    def __init__(self):
        self.loss = []
        self.val_loss = []
        pass

    def forward(self, A):
        A -= np.max(A)   # オーバーフロー対策
        self.Z = np.exp(A) / np.sum(np.exp(A), axis=1).reshape(-1, 1)
        return self.Z

    def backward(self, Z, Y):
        dA = Z - Y
        self.cross_entropy_loss(Z, Y)
        return dA
    
    def cross_entropy_loss(self, Z, Y, val=False):
        batch_size = Z.shape[0]
        delta = 1e-7
        loss = - np.sum(Y * np.log(Z + delta)) / batch_size
        if not val:
            self.loss.append(loss)
        else:
            self.val_loss.append(loss)
        return loss


class Tanh:
    def __init__(self):
        pass

    def __call__(self, A):
        self.Z = np.tanh(A)
        return self.Z

    def backward(self, dZ):
        dA = dZ * (1 - dZ**2)
        return dA

## 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\
h_t = tanh(a_t)
$$

a
t
 : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


h
t
 : 時刻tの状態・出力 (batch_size, n_nodes)


x
t
 : 時刻tの入力 (batch_size, n_features)


W
x
 : 入力に対する重み (n_features, n_nodes)


h
t
−
1
 : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


W
h
 : 状態に対する重み。 (n_nodes, n_nodes)


B
 : バイアス項 (n_nodes,)


初期状態 
h
0
 は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
x
 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

## 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。



```
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
```

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

```
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```

### RNN実行

In [ ]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
rnn = ScratchSimpleRNNClassifier()
rnn.fit(x)

array([[[0.76188798, 0.76213958, 0.76239095, 0.76255841],
        [0.792209  , 0.8141834 , 0.83404912, 0.84977719],
        [0.79494228, 0.81839002, 0.83939649, 0.85584174]]])

## 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$
W_x^{\prime} = W_x - \alpha \frac{\partial L}{\partial W_x} \\
W_h^{\prime} = W_h - \alpha \frac{\partial L}{\partial W_h} \\
B^{\prime} = B - \alpha \frac{\partial L}{\partial B}
$$

α
  : 学習率


∂
L
∂
W
x
 : 
W
x
 に関する損失 
L
 の勾配


∂
L
∂
W
h
 : 
W
h
 に関する損失 
L
 の勾配


∂
L
∂
B
 : 
B
 に関する損失 
L
 の勾配


勾配を求めるためのバックプロパゲーションの数式が以下です。


∂
h
t
∂
a
t
=
∂
L
∂
h
t
×
(
1
−
t
a
n
h
2
(
a
t
)
)

∂
L
∂
B
=
∂
h
t
∂
a
t

∂
L
∂
W
x
=
x
T
t
⋅
∂
h
t
∂
a
t

∂
L
∂
W
h
=
h
T
t
−
1
⋅
∂
h
t
∂
a
t

＊
∂
L
∂
h
t
 は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。


前の時刻や層に流す誤差の数式は以下です。


∂
L
∂
h
t
−
1
=
∂
h
t
∂
a
t
⋅
W
T
h

∂
L
∂
x
t
=
∂
h
t
∂
a
t
⋅
W
T
x

In [27]:
dA = np.array([[[-0.02149633,  0.00632246,  0.01809524, -0.01156441],
        [-0.09329432,  0.02335331, -0.00404074, -0.02144627],
        [-0.21688441, -0.01943171,  0.02462611, -0.01815572]]])
rnn2 = ScratchSimpleRNNClassifier()
rnn2.fit(x)
rnn2.backward(dA)

■ dHt_1 ----------------------------------
 [[[0. 0. 0. 0.]]]
■ dHt ----------------------------------
 [[[-0.21688441 -0.01943171  0.02462611 -0.01815572]]]
■ dA ----------------------------------
 [[[-0.02149633  0.00632246  0.01809524 -0.01156441]
  [-0.09329432  0.02335331 -0.00404074 -0.02144627]
  [-0.21688441 -0.01943171  0.02462611 -0.01815572]]]
■ dAt ----------------------------------
 [[[-0.20699402 -0.01942437  0.02461118 -0.01814974]]]
■ dWx ----------------------------------
 [[[-0.00620982 -0.00058273  0.00073834 -0.00054449]
  [-0.00827976 -0.00077697  0.00098445 -0.00072599]]]
■ self.dWh ----------------------------------
 [[[-0.1645483  -0.01544126  0.01956447 -0.01442799]
  [-0.16940184 -0.01589671  0.02014155 -0.01485356]
  [-0.17375005 -0.01630475  0.02065854 -0.01523483]
  [-0.17715412 -0.01662419  0.02106328 -0.0155333 ]]]
■ dHt_1 ----------------------------------
 [[[-0.00269259 -0.00489216 -0.00691024 -0.0092913 ]]]
■ dXt ----------------------------------
 [[

array([[[-0.00081249, -0.00118667],
        [-0.00309621, -0.00515657],
        [-0.00269259, -0.00691024]]])